# Model Serving Function

In [1]:
import mlrun

In [2]:
%nuclio cmd -c pip install lightgbm

In [3]:
%nuclio config spec.build.baseImage = "mlrun/mlrun"
%nuclio config kind = "serving"

%nuclio: setting spec.build.baseImage to 'mlrun/mlrun'
%nuclio: setting kind to 'serving'


In [4]:
import numpy as np
from cloudpickle import load

class LGBMModel(mlrun.serving.V2ModelServer):
    
    def load(self):
        model_file, extra_data = self.get_model('.pkl')
        self.model = load(open(model_file, 'rb'))

    def predict(self, body):
        try:
            feats = np.asarray(body['inputs'])
            result = self.model.predict(feats)
            return result.tolist()
        except Exception as exc:
            raise Exception(f"Failed to predict {exc}")

In [5]:
# mlrun: end-code

## Deploy and Test The Function

In [6]:
models_path = 'https://s3.wasabisys.com/iguazio/models/lightgbm/SampleModel.pkl'

In [7]:
fn = mlrun.code_to_function('lightgbm-serving',
                            description="LightGBM Serving",
                            categories=['serving', 'ml'],
                            labels={'author': 'edmondg', 'framework': 'lightgbm'})
fn.spec.default_class = 'LGBMModel'

In [8]:
fn.add_model('nyc-taxi-server', model_path=models_path)

In [9]:
# deploy the function
fn.apply(mlrun.platforms.auto_mount())
address = fn.deploy()

> 2021-01-28 10:56:00,391 [info] Starting remote function deploy
2021-01-28 10:56:00  (info) Deploying function
2021-01-28 10:56:00  (info) Building
2021-01-28 10:56:00  (info) Staging files and preparing base images
2021-01-28 10:56:00  (info) Building processor image
2021-01-28 10:56:01  (info) Build complete
2021-01-28 10:56:09  (info) Function deploy complete
> 2021-01-28 10:56:10,399 [info] function deployed, address=default-tenant.app.jinkwubtllaf.iguazio-cd1.com:32169


In [10]:
# test the function
my_data = '''{"inputs":[[5.1, 3.5, 1.4, 3, 5.1, 3.5, 1.4, 0.2, 5.1, 3.5, 1.4, 0.2, 5.1, 3.5, 1.4, 0.2]]}'''
fn.invoke('/v2/models/nyc-taxi-server/predict', my_data)

{'id': 'cd3aca51-1823-453b-81fb-e2cf2a148906',
 'model_name': 'nyc-taxi-server',
 'outputs': [25.374309065093435]}